In [ ]:
#default_exp web.app

# Web App

> Factory function that creates and configures the flask app.

In [ ]:
#export
from vulcan_medication_bundle.per_patient import *
from flask import Flask, g, request
from werkzeug.exceptions import abort
from pathlib import Path
import os

In [ ]:
from io import StringIO
import requests

In [ ]:
#demo
import pandas as pd

In [ ]:
#export
def get_single_patient_medication_bundle(api_base, subject):
    bundle = create_single_patient_medication_bundle(api_base, subject)
    bundle = handle_entry_search(bundle)
    bundle = filter_bundle(bundle, medication_status_filter)
    bundle = filter_bundle(bundle, do_not_perform_filter)
    return bundle

In [ ]:
#export
def create_app(test_config=None):
    "Create and configure an instance of the Flask application."
    app = Flask(__name__, instance_relative_config=True)
    app.config.from_mapping(
        # a default secret that should be overridden by instance config
        SECRET_KEY="dev",
        DATA_DIR=app.instance_path # used by filesystem and DB service
    )

    if test_config is None:
        # load the instance config, if it exists, when not testing
        app.config.from_pyfile("config.py", silent=True)
    else:
        # load the test config if passed in
        app.config.update(test_config)

    # ensure the instance folder exists
    os.makedirs(app.config['DATA_DIR'], exist_ok=True)
    
    @app.route("/hello")
    def hello(): return "Hello, World!"
    
    @app.route("/single_patient_medication_bundle")
    def _get_single_patient_medication_bundle():
        args = [request.args.get(p) for p in ['api_base', 'subject'] if request.args.get(p) is not None]
        if len(args) != 2:
            abort(400, 'Missing URL parameter: api_base and subject are required')
        return get_single_patient_medication_bundle(*args)

    return app

In [ ]:
assert not create_app().testing
assert create_app({"TESTING": True}).testing

In [ ]:
#hide
from flask import Flask
from contextlib import contextmanager
from pathlib import Path
import tempfile, shutil, os, vulcan_medication_bundle.web

def cd(p):
    print('Changing working directory:') # TODO: maybe we want to suppress stdout
    get_ipython().run_line_magic('cd', str(p))

@contextmanager
def test_resources():
    temp_path = tempfile.mkdtemp()
    try:
        cwd = get_ipython().run_line_magic('pwd', '')
        cd(Path(vulcan_medication_bundle.web.__file__).parent)
        app = create_app({"TESTING": True, "DATA_DIR": temp_path})
        client = app.test_client()
        with app.app_context():
            yield app, client
    finally:
        cd(cwd)
        shutil.rmtree(temp_path)

In [ ]:
with test_resources() as (app, client):
    response = client.get("/hello")
    assert response.data == b"Hello, World!"

Changing working directory:
C:\Users\Butterp\github\pete88b\vulcan_medication_bundle\vulcan_medication_bundle\web
Changing working directory:
C:\Users\Butterp\github\pete88b\vulcan_medication_bundle


In [ ]:
#demo
with test_resources() as (app, client):
    bundle_response = client.get("/single_patient_medication_bundle?api_base=http://hapi.fhir.org/baseR4&subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2")
cdisc_response = requests.post('https://mylinks-prod-sdtmtool.azurewebsites.net/TransformBundle/Process', json=bundle_response.json)
pd.read_csv(StringIO(cdisc_response.json()['cmcsv']))

Changing working directory:
C:\Users\Butterp\github\pete88b\vulcan_medication_bundle\vulcan_medication_bundle\web
GET http://hapi.fhir.org/baseR4/Patient?_id=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationRequest?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationDispense?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationAdministration?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/MedicationStatement?subject=0c4a1143-8d1c-42ed-b509-eac97d77c9b2
GET http://hapi.fhir.org/baseR4/Condition?_id=1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8
GET http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8
Changing working directory:
C:\Users\Butterp\github\pete88b\vulcan_medication_bundle


,STUDYID,DOMAIN,USUBJID,CMSEQ,CMGRPID,CMSPID,CMTRT,CMMODIFY,CMDECOD,CMCAT,...,CMENDTC,CMSTDY,CMENDY,CMDUR,CMSTRF,CMENRF,CMSTRTPT,CMSTTPT,CMENRTPT,CMENTPT
0,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,1,1,SPONSOR-1,Acetaminophen 500 MG Oral Tablet,NaN,NaN,CAT1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,2,1,SPONSOR-2,Acetaminophen 500 MG Oral Tablet,NaN,A:,CAT1,...,2020-01-14,13.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RWD-STUDY-01,CM,RWD-SUBJECT-01-30,3,1,SPONSOR-3,Acetaminophen 500 MG Oral Tablet,NaN,S:,CAT1,...,2020-01-14,13.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
